In [1]:
from neural_cherche import utils
from nltk.corpus import stopwords

stop_words = list(stopwords.words(fileids="english"))

documents, queries_ids, queries, qrels = utils.load_beir(
    dataset_name="scifact",
    split="test",
)

/Users/raphael/miniforge3/envs/antilopa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 5183/5183 [00:00<00:00, 273874.69it/s]


In [2]:
from neural_cherche import retrieve

from lenlp import sparse

count_vectorizer = sparse.BM25Vectorizer(
    normalize=True,
    ngram_range=(3, 5),
    analyzer="char_wb",
    stop_words=stop_words,
)

retriever = retrieve.TfIdf(
    key="id",
    on=["title", "text"],
    tfidf=count_vectorizer,
)

documents_embeddings = retriever.encode_documents(
    documents=documents,
)

retriever = retriever.add(documents_embeddings=documents_embeddings)

queries_embeddings = retriever.encode_queries(
    queries=queries,
)

scores = retriever(
    queries_embeddings=queries_embeddings,
    k=10,
    batch_size=1024,
    tqdm_bar=False,
)

In [3]:
utils.evaluate(
    scores=scores,
    qrels=qrels,
    queries_ids=queries_ids,
    metrics=["ndcg@10"] + [f"hits@{k}" for k in range(1, 11)],
)

{'ndcg@10': 0.6937352011710838,
 'hits@1': 0.5666666666666667,
 'hits@2': 0.68,
 'hits@3': 0.7466666666666667,
 'hits@4': 0.8166666666666667,
 'hits@5': 0.8466666666666667,
 'hits@6': 0.8733333333333333,
 'hits@7': 0.9033333333333333,
 'hits@8': 0.9033333333333333,
 'hits@9': 0.9166666666666666,
 'hits@10': 0.9233333333333333}

In [12]:
{
    "ndcg@10": 0.6947898732051012,
    "hits@1": 0.5633333333333334,
    "hits@2": 0.68,
    "hits@3": 0.7566666666666667,
    "hits@4": 0.8166666666666667,
    "hits@5": 0.8433333333333334,
    "hits@6": 0.8733333333333333,
    "hits@7": 0.9,
    "hits@8": 0.9066666666666666,
    "hits@9": 0.9166666666666666,
    "hits@10": 0.93,
}

151 ms ± 13.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


{'ndcg@10': 0.6805801205908164,
 'hits@1': 0.5633333333333334,
 'hits@2': 0.6733333333333333,
 'hits@3': 0.7433333333333333,
 'hits@4': 0.7733333333333333,
 'hits@5': 0.81,
 'hits@6': 0.8333333333333334,
 'hits@7': 0.8633333333333333,
 'hits@8': 0.87,
 'hits@9': 0.8833333333333333,
 'hits@10': 0.8966666666666666}

In [15]:
from neural_cherche import models, rank

model = models.ColBERT(
    model_name_or_path="raphaelsty/neural-cherche-colbert",
    device="mps",
)

ranker = rank.ColBERT(
    key="id",
    on=["title", "text"],
    model=model,
)

In [16]:
queries_embeddings = retriever.encode_queries(
    queries=queries,
)

candidates = retriever(
    queries_embeddings=queries_embeddings,
    k=30,
    batch_size=1024,
    tqdm_bar=False,
)

ranker_queries_embeddings = ranker.encode_queries(queries=queries, batch_size=32)

ranker_documents_embeddings = ranker.encode_candidates_documents(
    documents=documents, candidates=candidates, batch_size=32
)

ColBERT documents embeddings: 100%|██████████| 122/122 [01:33<00:00,  1.31it/s]


In [17]:
scores = ranker(
    documents=candidates,
    queries_embeddings=ranker_queries_embeddings,
    documents_embeddings=ranker_documents_embeddings,
    k=10,
    batch_size=32,
)

ColBERT ranker:  99%|█████████▉| 299/301 [00:02<00:00, 126.34it/s]


{'ndcg@10': 0.7120634760361798,
 'hits@1': 0.5933333333333334,
 'hits@2': 0.7066666666666667,
 'hits@3': 0.7766666666666666,
 'hits@4': 0.85,
 'hits@5': 0.8633333333333333,
 'hits@6': 0.8933333333333333,
 'hits@7': 0.9133333333333333,
 'hits@8': 0.9233333333333333,
 'hits@9': 0.93,
 'hits@10': 0.94}

In [21]:
# MS MARCO, 8.84M documents
# 5 minutes 25 seconds to create the index
# 118 queries per second on 8.84M documents

# 6,825,631 articles in wikipedia
# Assume wikipedia articles are 100 times longer than MS MARCO documents
# 300 minutes to create the index for wikipedia on a single computer,
# Could be 150 minutes

In [3]:
wikipedia = """
Main menu
 
WikipediaThe Free Encyclopedia

Search
Create account
Log in

Personal tools
Contents  hide
(Top)
Design
Toggle Design subsection
Pretraining
Architecture details
Performance
Analysis
History
Recognition
References
Further reading
External links
BERT (language model)

16 languages
Article
Talk
Read
Edit
View history

Tools
From Wikipedia, the free encyclopedia
Bidirectional Encoder Representations from Transformers (BERT) is a language model based on the transformer architecture, notable for its dramatic improvement over previous state of the art models. It was introduced in October 2018 by researchers at Google.[1][2] A 2020 literature survey concluded that "in a little over a year, BERT has become a ubiquitous baseline in Natural Language Processing (NLP) experiments counting over 150 research publications analyzing and improving the model."[3]
BERT was originally implemented in the English language at two model sizes:[1] (1) BERTBASE: 12 encoders with 12 bidirectional self-attention heads totaling 110 million parameters, and (2) BERTLARGE: 24 encoders with 16 bidirectional self-attention heads totaling 340 million parameters. Both models were pre-trained on the Toronto BookCorpus[4] (800M words) and English Wikipedia (2,500M words).
Design
BERT is an "encoder-only" transformer architecture.
On a high level, BERT consists of three modules:
embedding. This module converts an array of one-hot encoded tokens into an array of vectors representing the tokens.
a stack of encoders. These encoders are the Transformer encoders. They perform transformations over the array of representation vectors.
un-embedding. This module converts the final representation vectors into one-hot encoded tokens again.
The un-embedding module is necessary for pretraining, but it is often unnecessary for downstream tasks. Instead, one would take the representation vectors output at the end of the stack of encoders, and use those as a vector representation of the text input, and train a smaller model on top of that.
BERT uses WordPiece to convert each English word into an integer code. Its vocabulary has size 30,000. Any token not appearing in its vocabulary is replaced by [UNK] for "unknown".
Pretraining
BERT was pre-trained simultaneously on two tasks:[5]
language modeling: 15% of tokens were selected for prediction, and the training objective was to predict the selected token given its context. The selected token is
replaced with a [MASK] token with probability 80%,
replaced with a random word token with probability 10%,
not replaced with probability 10%.
For example, the sentence "my dog is cute" may have the 4-th token selected for prediction. The model would have input text
"my dog is [MASK]" with probability 80%,
"my dog is happy" with probability 10%,
"my dog is cute" with probability 10%.
After processing the input text, the model's 4-th output vector is passed to a separate neural network, which outputs a probability distribution over its 30,000-large vocabulary.
next sentence prediction: Given two spans of text, the model predicts if these two spans appeared sequentially in the training corpus, outputting either [IsNext] or [NotNext]. The first span starts with a special token [CLS] (for "classify"). The two spans are separated by a special token [SEP] (for "separate"). After processing the two spans, the 1-st output vector (the vector coding for [CLS]) is passed to a separate neural network for the binary classification into [IsNext] and [NotNext].
For example, given "[CLS] my dog is cute [SEP] he likes playing" the model should output token [IsNext].
Given "[CLS] my dog is cute [SEP] how do magnets work" the model should output token [NotNext].
As a result of this training process, BERT learns latent representations of words and sentences in context. After pre-training, BERT can be fine-tuned with fewer resources on smaller datasets to optimize its performance on specific tasks such as NLP tasks (language inference, text classification) and sequence-to-sequence based language generation tasks (question-answering, conversational response generation).[1][6] The pre-training stage is significantly more computationally expensive than fine-tuning.
Architecture details
This section describes BERTBASE. The other one, BERTLARGE, is similar, just larger.
The lowest layer is the embedding layer, which contains three components: word_embeddings, position_embeddings, token_type_embeddings.
word_embeddings takes in a one-hot vector of the input token. The one-hot vector input has dimension 30,000, because BERT has a vocabulary size that large.
position_embeddings performs absolute position embedding. It is like word_embeddings, but on a vocabulary consisting of just the time-stamps 0 to 511, since BERT has a context window of 512.
token_type_embeddings is like word_embeddings, but on a vocabulary consisting of just 0 and 1. The only type-1 tokens are those that appear after the [SEP]. All other tokens are type-0.
The three outputs are added, then pushed through a LayerNorm (layer normalization), obtaining an array of representation vectors, each having 768 dimensions.
After this, the representation vectors move through 12 Transformer encoders, then they are un-embedded by an affine-Add & LayerNorm-linear.
Performance
When BERT was published, it achieved state-of-the-art performance on a number of natural language understanding tasks:[1]
GLUE (General Language Understanding Evaluation) task set (consisting of 9 tasks)
SQuAD (Stanford Question Answering Dataset[7]) v1.1 and v2.0
SWAG (Situations With Adversarial Generations[8])
Analysis
The reasons for BERT's state-of-the-art performance on these natural language understanding tasks are not yet well understood.[9][10] Current research has focused on investigating the relationship behind BERT's output as a result of carefully chosen input sequences,[11][12] analysis of internal vector representations through probing classifiers,[13][14] and the relationships represented by attention weights.[9][10] The high performance of the BERT model could also be attributed to the fact that it is bidirectionally trained. This means that BERT, based on the Transformer model architecture, applies its self-attention mechanism to learn information from a text from the left and right side during training, and consequently gains a deep understanding of the context. For example, the word fine can have two different meanings depending on the context (I feel fine today, She has fine blond hair). BERT considers the words surrounding the target word fine from the left and right side.
However it comes at a cost: due to encoder-only architecture lacking a decoder, BERT can't be prompted and can't generate text, while bidirectional models in general do not work effectively without the right side,[clarification needed] thus being difficult to prompt, with even short text generation requiring sophisticated computationally expensive techniques.[15]
In contrast to deep learning neural networks which require very large amounts of data, BERT has already been pre-trained which means that it has learnt the representations of the words and sentences as well as the underlying semantic relations that they are connected with. BERT can then be fine-tuned on smaller datasets for specific tasks such as sentiment classification. The pre-trained models are chosen according to the content of the given dataset one uses but also the goal of the task. For example, if the task is a sentiment classification task on financial data, a pre-trained model for the analysis of sentiment of financial text should be chosen. The weights of the original pre-trained models were released on GitHub.[16]
History
BERT was originally published by Google researchers Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova. The design has its origins from pre-training contextual representations, including semi-supervised sequence learning,[17] generative pre-training, ELMo,[18] and ULMFit.[19] Unlike previous models, BERT is a deeply bidirectional, unsupervised language representation, pre-trained using only a plain text corpus. Context-free models such as word2vec or GloVe generate a single word embedding representation for each word in the vocabulary, whereas BERT takes into account the context for each occurrence of a given word. For instance, whereas the vector for "running" will have the same word2vec vector representation for both of its occurrences in the sentences "He is running a company" and "He is running a marathon", BERT will provide a contextualized embedding that will be different according to the sentence.[citation needed]
On October 25, 2019, Google announced that they had started applying BERT models for English language search queries within the US.[20] On December 9, 2019, it was reported that BERT had been adopted by Google Search for over 70 languages.[21] In October 2020, almost every single English-based query was processed by a BERT model.[22]
A later paper proposes RoBERTa, which preserves BERT's architecture, but improves its training, changing key hyperparameters, removing the next-sentence prediction task, and using much larger mini-batch sizes.[23]
Recognition
The research paper describing BERT won the Best Long Paper Award at the 2019 Annual Conference of the North American Chapter of the Association for Computational Linguistics (NAACL).[24]
References
 Devlin, Jacob; Chang, Ming-Wei; Lee, Kenton; Toutanova, Kristina (October 11, 2018). "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding". arXiv:1810.04805v2 [cs.CL].
 "Open Sourcing BERT: State-of-the-Art Pre-training for Natural Language Processing". Google AI Blog. November 2, 2018. Retrieved November 27, 2019.
 Rogers, Anna; Kovaleva, Olga; Rumshisky, Anna (2020). "A Primer in BERTology: What We Know About How BERT Works". Transactions of the Association for Computational Linguistics. 8: 842–866. arXiv:2002.12327. doi:10.1162/tacl_a_00349. S2CID 211532403.
 Zhu, Yukun; Kiros, Ryan; Zemel, Rich; Salakhutdinov, Ruslan; Urtasun, Raquel; Torralba, Antonio; Fidler, Sanja (2015). "Aligning Books and Movies: Towards Story-Like Visual Explanations by Watching Movies and Reading Books". pp. 19–27. arXiv:1506.06724 [cs.CV].
 "Summary of the models — transformers 3.4.0 documentation". huggingface.co. Retrieved February 16, 2023.
 Horev, Rani (2018). "BERT Explained: State of the art language model for NLP". Towards Data Science. Retrieved September 27, 2021.
 Rajpurkar, Pranav; Zhang, Jian; Lopyrev, Konstantin; Liang, Percy (October 10, 2016). "SQuAD: 100,000+ Questions for Machine Comprehension of Text". arXiv:1606.05250 [cs.CL].
 Zellers, Rowan; Bisk, Yonatan; Schwartz, Roy; Choi, Yejin (August 15, 2018). "SWAG: A Large-Scale Adversarial Dataset for Grounded Commonsense Inference". arXiv:1808.05326 [cs.CL].
 Kovaleva, Olga; Romanov, Alexey; Rogers, Anna; Rumshisky, Anna (November 2019). "Revealing the Dark Secrets of BERT". Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th International Joint Conference on Natural Language Processing (EMNLP-IJCNLP). pp. 4364–4373. doi:10.18653/v1/D19-1445. S2CID 201645145.
 Clark, Kevin; Khandelwal, Urvashi; Levy, Omer; Manning, Christopher D. (2019). "What Does BERT Look at? An Analysis of BERT's Attention". Proceedings of the 2019 ACL Workshop BlackboxNLP: Analyzing and Interpreting Neural Networks for NLP. Stroudsburg, PA, USA: Association for Computational Linguistics: 276–286. arXiv:1906.04341. doi:10.18653/v1/w19-4828.
 Khandelwal, Urvashi; He, He; Qi, Peng; Jurafsky, Dan (2018). "Sharp Nearby, Fuzzy Far Away: How Neural Language Models Use Context". Proceedings of the 56th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers). Stroudsburg, PA, USA: Association for Computational Linguistics: 284–294. arXiv:1805.04623. doi:10.18653/v1/p18-1027. S2CID 21700944.
 Gulordava, Kristina; Bojanowski, Piotr; Grave, Edouard; Linzen, Tal; Baroni, Marco (2018). "Colorless Green Recurrent Networks Dream Hierarchically". Proceedings of the 2018 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long Papers). Stroudsburg, PA, USA: Association for Computational Linguistics. pp. 1195–1205. arXiv:1803.11138. doi:10.18653/v1/n18-1108. S2CID 4460159.
 Giulianelli, Mario; Harding, Jack; Mohnert, Florian; Hupkes, Dieuwke; Zuidema, Willem (2018). "Under the Hood: Using Diagnostic Classifiers to Investigate and Improve how Language Models Track Agreement Information". Proceedings of the 2018 EMNLP Workshop BlackboxNLP: Analyzing and Interpreting Neural Networks for NLP. Stroudsburg, PA, USA: Association for Computational Linguistics: 240–248. arXiv:1808.08079. doi:10.18653/v1/w18-5426. S2CID 52090220.
 Zhang, Kelly; Bowman, Samuel (2018). "Language Modeling Teaches You More than Translation Does: Lessons Learned Through Auxiliary Syntactic Task Analysis". Proceedings of the 2018 EMNLP Workshop BlackboxNLP: Analyzing and Interpreting Neural Networks for NLP. Stroudsburg, PA, USA: Association for Computational Linguistics: 359–361. doi:10.18653/v1/w18-5448.
 Patel, Ajay; Li, Bryan; Mohammad Sadegh Rasooli; Constant, Noah; Raffel, Colin; Callison-Burch, Chris (2022). "Bidirectional Language Models Are Also Few-shot Learners". arXiv:2209.14500 [cs.LG].
 "BERT". GitHub. Retrieved March 28, 2023.
 Dai, Andrew; Le, Quoc (November 4, 2015). "Semi-supervised Sequence Learning". arXiv:1511.01432 [cs.LG].
 Peters, Matthew; Neumann, Mark; Iyyer, Mohit; Gardner, Matt; Clark, Christopher; Lee, Kenton; Luke, Zettlemoyer (February 15, 2018). "Deep contextualized word representations". arXiv:1802.05365v2 [cs.CL].
 Howard, Jeremy; Ruder, Sebastian (January 18, 2018). "Universal Language Model Fine-tuning for Text Classification". arXiv:1801.06146v5 [cs.CL].
 Nayak, Pandu (October 25, 2019). "Understanding searches better than ever before". Google Blog. Retrieved December 10, 2019.
 Montti, Roger (December 10, 2019). "Google's BERT Rolls Out Worldwide". Search Engine Journal. Retrieved December 10, 2019.
 "Google: BERT now used on almost every English query". Search Engine Land. October 15, 2020. Retrieved November 24, 2020.
 Liu, Yinhan; Ott, Myle; Goyal, Naman; Du, Jingfei; Joshi, Mandar; Chen, Danqi; Levy, Omer; Lewis, Mike; Zettlemoyer, Luke; Stoyanov, Veselin (2019). "RoBERTa: A Robustly Optimized BERT Pretraining Approach". arXiv:1907.11692 [cs.CL].
 "Best Paper Awards". NAACL. 2019. Retrieved March 28, 2020.
Further reading
Rogers, Anna; Kovaleva, Olga; Rumshisky, Anna (2020). "A Primer in BERTology: What we know about how BERT works". arXiv:2002.12327 [cs.CL].
External links
Official GitHub repository
BERT on Devopedia
vte
Google AI
vte
Google
vte
Natural language processing
vte
Differentiable computing
Categories: Google softwareLarge language models
This page was last edited on 7 May 2024, at 09:13 (UTC).
Text is available under the Creative Commons Attribution-ShareAlike License 4.0; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia® is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization.
Privacy policyAbout WikipediaDisclaimersContact WikipediaCode of ConductDevelopersStatisticsCookie statementMobile view
Wikimedia FoundationPowered by MediaWiki
Toggle limited content width
"""

In [15]:
from nltk.corpus import stopwords

from lenlp import flash

stop_words = list(stopwords.words(fileids="english"))
flash_text = flash.FlashText(normalize=True)
flash_text = flash_text.add(x=stop_words)

In [16]:
%%timeit 
extract = flash_text.extract(x=[wikipedia] * 1000, span_info=True)

681 ms ± 20.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
from flashtext import KeywordProcessor

flash_text = KeywordProcessor()
flash_text.add_keywords_from_list(stop_words)

In [20]:
%%timeit 
extract = []
for page in [wikipedia] * 1000:
    extract.append(flash_text.extract_keywords(page))

1.33 s ± 23.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
from neural_cherche import retrieve

from lenlp import sparse

count_vectorizer = sparse.CountVectorizer(
    normalize=True, ngram_range=(3, 5), analyzer="char_wb", stop_words=stop_words
)

retriever = retrieve.BM25(
    key="id",
    on=["title", "text"],
    count_vectorizer=count_vectorizer,
)

documents_embeddings = retriever.encode_documents(
    documents=documents,
)

retriever = retriever.add(documents_embeddings=documents_embeddings)

/Users/raphael/miniforge3/envs/antilopa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'documents' is not defined

In [38]:
queries_embeddings = retriever.encode_queries(
    queries=queries,
)

scores = retriever(
    queries_embeddings=queries_embeddings,
    k=10,
    batch_size=1024,
    tqdm_bar=False,
)

utils.evaluate(
    scores=scores,
    qrels=qrels,
    queries_ids=queries_ids,
    metrics=["ndcg@10"] + [f"hits@{k}" for k in range(1, 11)],
)

{'ndcg@10': 0.6805801205908164,
 'hits@1': 0.5633333333333334,
 'hits@2': 0.6733333333333333,
 'hits@3': 0.7433333333333333,
 'hits@4': 0.7733333333333333,
 'hits@5': 0.81,
 'hits@6': 0.8333333333333334,
 'hits@7': 0.8633333333333333,
 'hits@8': 0.87,
 'hits@9': 0.8833333333333333,
 'hits@10': 0.8966666666666666}

In [34]:
from neural_cherche import retrieve
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(
    lowercase=True, ngram_range=(3, 5), analyzer="char_wb", strip_accents="unicode"
)

retriever = retrieve.BM25(
    key="id",
    on=["title", "text"],
    count_vectorizer=count_vectorizer,
)

documents_embeddings = retriever.encode_documents(
    documents=documents,
)

retriever = retriever.add(documents_embeddings=documents_embeddings)

In [36]:
queries_embeddings = retriever.encode_queries(
    queries=queries,
)

scores = retriever(
    queries_embeddings=queries_embeddings,
    k=10,
    batch_size=1024,
    tqdm_bar=False,
)
utils.evaluate(
    scores=scores,
    qrels=qrels,
    queries_ids=queries_ids,
    metrics=["ndcg@10"] + [f"hits@{k}" for k in range(1, 11)],
)

{'ndcg@10': 0.6391469805305356,
 'hits@1': 0.51,
 'hits@2': 0.62,
 'hits@3': 0.68,
 'hits@4': 0.74,
 'hits@5': 0.7866666666666666,
 'hits@6': 0.8066666666666666,
 'hits@7': 0.8266666666666667,
 'hits@8': 0.85,
 'hits@9': 0.8633333333333333,
 'hits@10': 0.8733333333333333}

In [ ]:
{
    "ndcg@10": 0.6805801205908164,
    "hits@1": 0.5633333333333334,
    "hits@2": 0.6733333333333333,
    "hits@3": 0.7433333333333333,
    "hits@4": 0.7733333333333333,
    "hits@5": 0.81,
    "hits@6": 0.8333333333333334,
    "hits@7": 0.8633333333333333,
    "hits@8": 0.87,
    "hits@9": 0.8833333333333333,
    "hits@10": 0.8966666666666666,
}

In [5]:
import numpy as np
from scipy.sparse import csr_matrix


def normalize_csr_matrix(csr_mat):
    # Calculate the L2 norm for each row
    row_norms = np.sqrt(csr_mat.multiply(csr_mat).sum(axis=1))

    # Avoid division by zero by setting zero norms to one (rows with all zeros)
    row_norms[row_norms == 0] = 1

    # Normalize each row by its L2 norm
    row_indices, col_indices = csr_mat.nonzero()
    for i, j in zip(row_indices, col_indices):
        csr_mat[i, j] /= row_norms[i, 0]

    return csr_mat


# Example usage:
# Create a sample CSR matrix
sample_data = np.array([1, 2, 3, 4, 5, 6])
sample_indices = np.array([0, 2, 2, 0, 1, 2])
sample_indptr = np.array([0, 2, 3, 6])
sample_csr_matrix = csr_matrix(
    (sample_data, sample_indices, sample_indptr), shape=(3, 3)
)

# Normalize the matrix
normalized_csr_matrix = normalize_csr_matrix(sample_csr_matrix)

# Convert to dense format to check the result (optional)
print(normalized_csr_matrix.todense())

[[0 0 0]
 [0 0 1]
 [0 0 0]]


In [6]:
sample_csr_matrix.todense()

matrix([[0, 0, 0],
        [0, 0, 1],
        [0, 0, 0]])